In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Prepare High-Quality Preference Data for Gemini 2.5

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Ftuning%2Fdpo_gemini_data_prep_tuning.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb">
      <img width="32px" src="https://storage.googleapis.com/github-repo/generative-ai/logos/GitHub_Invertocat_Dark.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini_data_prep_tuning.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>    

| Authors |
| --- |
| Haichao Yu |
| [Ivan Nardini](https://github.com/inardini) |

## Overview

This tutorial teaches you how to prepare high-quality preference data for tuning Gemini 2.5. Before running Direct Preference Optimization (DPO), it's crucial to ensure your training data is high quality. We will use **Vertex AI Gen AI Evaluation SDK** to filter out low-quality preference pairs.

### Why does data quality matter?

Imagine you're teaching by showing examples. If half your "good examples" are actually mediocre, and half your "bad examples" are actually decent, you'll confuse the student. The same happens when training AI models.

**Think of preference data like graded essays:**
- Some essays marked "A" might actually be B-quality
- Some essays marked "F" might actually deserve a C
- If you don't filter these out, your model learns the wrong patterns

### What you'll learn

By the end of this tutorial, you will:
1.  Load a preference dataset (UltraFeedback).
2.  Use **Vertex AI Gen AI Evaluation SDK** to score responses.
3.  Visualize quality scores to understand your data.
4.  Apply filtering strategies (Absolute Threshold and Win Margin).
5.  Create a Supervised Fine-Tuning (SFT) dataset from high-quality responses.

## Get started

### Install required packages

We'll install the necessary packages for Google Cloud, data handling, and evaluation.

**⚠️ Expected Behavior**: After running this cell, Colab will ask you to **restart the runtime**. This is normal and necessary.

In [ ]:
%pip install --upgrade --quiet "google-cloud-aiplatform[evaluation]" datasets matplotlib

### Authenticate (Colab only)

If you're running this in Google Colab, you need to authenticate to access your Google Cloud project.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("✅ Authentication successful!")

### Set up your Google Cloud project

Replace `[your-project-id]` below with your actual Google Cloud project ID.

In [ ]:
import os

from vertexai import Client, types

# TODO: Replace with your actual project ID
# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT", ""))

if not PROJECT_ID:
    raise ValueError("Please set your PROJECT_ID above")

LOCATION = "us-central1"
BUCKET_NAME = f"{PROJECT_ID}-data-prep"
BUCKET_URI = f"gs://{BUCKET_NAME}"

print(f"📦 Creating GCS bucket: {BUCKET_NAME}...")
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI} 2>/dev/null || echo "(Bucket already exists, continuing...)"

# Initialize the Client
client = Client(project=PROJECT_ID, location=LOCATION)

print(f"✅ Using project: {PROJECT_ID}")
print(f"✅ Bucket: {BUCKET_URI}")

### Import libraries

In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display
from datasets import load_dataset

print("✅ Libraries imported")

## Step 1: Load the training dataset

We'll use the **UltraFeedback** dataset, same as in the DPO tutorial. This dataset contains prompts with human-rated responses.

In [ ]:
print("📥 Loading UltraFeedback dataset...")
dataset = load_dataset("zhengr/ultrafeedback_binarized")

In [ ]:
# Define the Filter Function
def is_valid_example(example):
    """Returns True if the example has a valid prompt and valid responses."""
    # Check Prompt
    if not example.get("prompt"):
        return False

    # Check Chosen (Preferred) content
    # Ensure list exists, is not empty, and last message has content
    if not example.get("chosen") or not example["chosen"][-1].get("content"):
        return False

    # Check Rejected (Dispreferred) content
    if not example.get("rejected") or not example["rejected"][-1].get("content"):
        return False

    return True


# Apply Filter
print("🧹 Filtering empty items from dataset...")
# This removes invalid rows from the entire dataset split efficiently
clean_dataset = dataset["train_prefs"].filter(is_valid_example)

print(f"Original size: {len(dataset['train_prefs'])}")
print(f"Clean size:    {len(clean_dataset)}")

# Select Subset
# Now we select 100 items knowing they are all valid
# We use min() just in case the dataset is smaller than 100 after filtering
n_examples = 100
train_data = clean_dataset.select(range(min(n_examples, len(clean_dataset))))

print(f"✅ Loaded {len(train_data)} CLEAN examples for training.")

## Step 2: Data Quality Evaluation

We will use Vertex AI's Gen AI Evaluation SDK to score both preferred and dispreferred responses. This provides a streamlined way to run evaluations without manual API calls or parallelization.

### 2.1. Prepare Data for Evaluation

We need to extract the prompt and the response for each example. We'll create two dataframes: one for preferred responses and one for dispreferred. The SDK expects columns named `prompt` and `response`.

In [ ]:
def prepare_eval_data(data, subset="preferred"):
    prompts = []
    responses = []

    print(f"Processing subset: {subset}")

    for i, example in enumerate(data):
        # Extract content first
        current_prompt = example["prompt"]

        if subset == "preferred":
            resp = example["chosen"][-1]["content"]
        else:
            resp = example["rejected"][-1]["content"]

        # Append BOTH at the same time
        # This guarantees prompts and responses have the exact same length
        prompts.append(current_prompt)
        responses.append(resp)

    return pd.DataFrame({"prompt": prompts, "response": responses})


df_preferred = prepare_eval_data(train_data, "preferred")
df_dispreferred = prepare_eval_data(train_data, "dispreferred")

print("✅ Data prepared for evaluation.")
display(df_preferred.head(2))

### 2.2. Run Auto-Evaluation using Gen AI Evaluation SDK

We'll use the `client.evals.evaluate` method. We'll use the `general_quality_v1` metric.

In [ ]:
# Define the evaluation metric
metric_to_use = types.RubricMetric.GENERAL_QUALITY
metric_name_for_results = "general_quality_v1"

# Helper Function to Extract Scores


def extract_scores_from_result(result, metric_name):
    """Iterates through the EvaluationResult object to extract individual scores."""
    scores = []
    # Ensure we process them in order of their index to match the DataFrame
    sorted_cases = sorted(result.eval_case_results, key=lambda x: x.eval_case_index)

    for case in sorted_cases:
        try:
            # Get the result for the first candidate (we only evaluate 1 response per row)
            metric_result = case.response_candidate_results[0].metric_results[
                metric_name
            ]
            scores.append(metric_result.score)
        except (KeyError, IndexError, AttributeError):
            scores.append(None)  # Handle cases where evaluation might have failed

    return scores


print("Running evaluation on preferred responses...")
result_preferred = client.evals.evaluate(
    dataset=df_preferred,
    metrics=[metric_to_use],
)

df_preferred["score"] = extract_scores_from_result(
    result_preferred, metric_name_for_results
)

print("Running evaluation on dispreferred responses...")
result_dispreferred = client.evals.evaluate(
    dataset=df_dispreferred,
    metrics=[metric_to_use],
)
df_dispreferred["score"] = extract_scores_from_result(
    result_dispreferred, metric_name_for_results
)

print("✅ Evaluation complete.")
print()
display(df_preferred[["prompt", "score"]].head())

## Step 3: Visualize Quality Scores

Now for the interesting part! Let's visualize the quality scores to see how well your "preferred" and "dispreferred" labels match actual quality.

**What we're looking for:**
- **Good data**: Preferred responses score higher than dispreferred ones (clear separation in the histogram)
- **Problematic data**: Lots of overlap between the two distributions

This visualization will help you decide where to set filtering thresholds in the next step.

In [ ]:
# Create histogram
# Using 20 bins to get better granularity on the 0-1 scale
bins = np.linspace(0, 1, 20)

plt.figure(figsize=(10, 6))
plt.hist(
    df_preferred["score"].dropna(),
    bins,
    alpha=0.5,
    label=f"Preferred (avg: {df_preferred['score'].mean():.2f})",
    color="green",
)
plt.hist(
    df_dispreferred["score"].dropna(),
    bins,
    alpha=0.5,
    label=f"Dispreferred (avg: {df_dispreferred['score'].mean():.2f})",
    color="red",
)

# Axis labels reflect the 0-1 scale
plt.xlabel("Quality Score (0.0 = poor, 1.0 = perfect)")
plt.ylabel("Number of Responses")
plt.title("Distribution of Quality Scores: Preferred vs Dispreferred Responses")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Step 4: Data Cleaning (Filtering)

Now we'll filter the dataset based on the scores we just computed.

### Filtering Strategies

| Strategy | When to Use | How It Works |
|----------|-------------|--------------|
| **Absolute Threshold** | You want to set a minimum quality bar | Keep if preferred > threshold AND dispreferred < threshold |
| **Win Margin** | You want clear winners | Keep if preferred score is at least X points higher than dispreferred |

### Strategy 1: Absolute Threshold

We keep pairs where:
- Preferred score > 0.7 (out of 5)
- Dispreferred score < 0.5

In [ ]:
PREFERRED_THRESHOLD = 0.65
DISPREFERRED_THRESHOLD = 0.95

# Calculate how much data each rule removes.
remove_mask_abs_preferred = df_preferred["score"] <= PREFERRED_THRESHOLD
remove_mask_abs_dispreferred = df_dispreferred["score"] >= DISPREFERRED_THRESHOLD
keep_mask_abs = (df_preferred["score"] > PREFERRED_THRESHOLD) & (
    df_dispreferred["score"] < DISPREFERRED_THRESHOLD
)

print(
    f"Pairs removed with preferred response filtering only: {remove_mask_abs_preferred.sum()} out of {len(train_data)}"
)
print(
    f"Pairs removed with dispreferred response filtering only: {remove_mask_abs_dispreferred.sum()} out of {len(train_data)}"
)
print(
    f"Pairs kept with Absolute Threshold: {keep_mask_abs.sum()} out of {len(train_data)}"
)

### Strategy 2: Win Margin

We keep pairs where the preferred score is at least 1 point higher than the dispreferred score.

In [ ]:
MARGIN = 0.05
keep_mask_margin = (df_preferred["score"] - df_dispreferred["score"]) >= MARGIN

print(f"Pairs kept with Win Margin: {keep_mask_margin.sum()} out of {len(train_data)}")

### Save Cleaned Data

Let's save the cleaned data using the Win Margin strategy, transformed into Gemini format.

In [ ]:
cleaned_data = []
for i, keep in enumerate(keep_mask_margin):
    if keep:
        example = train_data[i]
        gemini_format = {
            "contents": [{"role": "user", "parts": [{"text": example["prompt"]}]}],
            "completions": [
                {
                    "score": 1.0,
                    "completion": {
                        "role": "model",
                        "parts": [{"text": example["chosen"][-1]["content"]}],
                    },
                },
                {
                    "score": 0.0,
                    "completion": {
                        "role": "model",
                        "parts": [{"text": example["rejected"][-1]["content"]}],
                    },
                },
            ],
        }
        cleaned_data.append(gemini_format)

output_file = "cleaned_preference_data.jsonl"
with open(output_file, "w") as f:
    for entry in cleaned_data:
        f.write(json.dumps(entry) + "\n")

!gsutil cp {output_file} {BUCKET_URI}/{output_file}
print(f"✅ Cleaned data saved to {BUCKET_URI}/{output_file}")

## Step 5: Create SFT Dataset (Optional)

**Bonus step!** You can extract just the preferred responses to create a supervised fine-tuning (SFT) dataset.

### Why would you want this?

The [preference optimization tutorial](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini.ipynb) recommends a **two-phase approach**:

1. **Phase 1: SFT (Supervised Fine-Tuning)** - Train the model on just the good responses
2. **Phase 2: DPO (Direct Preference Optimization)** - Show the model pairs to refine its judgment

**Think of it like teaching:**
- **SFT** = "Here are examples of good essays" (establishes baseline competence)
- **DPO** = "Now here's why this essay is better than that one" (refines judgment)

This two-phase approach often produces better results than jumping straight to preference optimization.

In [ ]:
sft_data = []
for entry in cleaned_data:
    # Extract preferred response (score 1.0)
    preferred_resp = next(
        c["completion"] for c in entry["completions"] if c["score"] == 1.0
    )
    sft_entry = {"contents": [entry["contents"][0], preferred_resp]}
    sft_data.append(sft_entry)

sft_output_file = "sft_data.jsonl"
with open(sft_output_file, "w") as f:
    for entry in sft_data:
        f.write(json.dumps(entry) + "\n")

!gsutil cp {sft_output_file} {BUCKET_URI}/{sft_output_file}
print(f"✅ SFT data saved to {BUCKET_URI}/{sft_output_file}")

## Conclusion

You have now prepared a high-quality preference dataset and an SFT dataset using the Vertex AI Gen AI Evaluation SDK. These are ready to be used in the [Gemini DPO Tutorial](https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/tuning/dpo_gemini.ipynb).

### Clean up

To avoid charges, you can delete the GCS bucket created in this tutorial.

In [ ]:
# !gsutil rm -r {BUCKET_URI}